In [1]:
import numpy as np 
import pandas as pd 
import shutil
import os
import zipfile
import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
import copy
import tqdm
from PIL import Image
from albumentations import pytorch as AT
import albumentations as A
import torchvision.datasets as dataset
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import natsort
!pip install -U skorch
import time


import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split

from skorch import NeuralNetClassifier
from skorch.dataset import CVSplit
from skorch.callbacks import LRScheduler, Checkpoint 
from skorch.callbacks import Freezer, EarlyStopping

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

# for multiprocessing
import multiprocessing as mp


     |████████████████████████████████| 133kB 9.2MB/s 


In [2]:
class DatasetRetriever(Dataset):
  def __init__(self, dir,image_list,labels, transform=None,mode='train'):
        self.dir = dir
        self.labels = labels
        self.image_list=image_list
        self.label=0
        self.mode=mode
        self.image_name=None
        self.full_path=None
        self.transform=transform
        self.image=None

  def __len__(self):
        return len(self.image_list)

  def __getitem__(self, idx: int):       
        self.image_name = self.image_list[idx]
        self.full_path = os.path.join(self.dir, self.image_name)
        if self.mode=='train':
            #if self.image_name.split('.')[0] in self.labels['image'].unique():
                self.image = cv2.imread(self.full_path)
                self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
                self.label= int(self.labels[self.labels['image']==self.image_name.split('.')[0]]['class'].values[0])
        else:
            self.image = cv2.imread(self.full_path)
            self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=self.image)
            self.image = augmented['image']
        if self.mode == 'train':
            return self.image, self.label
        else:
            return self.image, self.image_name.split('.')[0]



In [3]:
class MobileNet(nn.Module):
    def __init__(self, output_features, num_units=512, drop=0.5,
                 num_units1=256, drop1=0.2):
        super().__init__()
        model =  torchvision.models.mobilenet_v2(pretrained=False)
        n_inputs = model.classifier[1].in_features
        model.classifier = nn.Sequential(
                                nn.Dropout(p=drop1), 
                                nn.Linear(n_inputs, output_features))
        self.model = model
        
    def forward(self, x):
        return self.model(x)


lr_scheduler_mobilenet = LRScheduler(policy='StepLR',
                                  step_size=8,gamma=0.2)
# callback for saving the best on validation accuracy model
checkpoint_mobilenet = Checkpoint(f_params='/content/drive/MyDrive/chess_weights/best_model_mobilenet.pkl',
                            monitor='valid_acc_best')
# callback for freezing all layer of the model except the last layer
#freezer_vgg = Freezer(lambda x: not x.startswith('model.classifier'))
# callback for early stopping
early_stopping_mobilenet = EarlyStopping(patience=10)
mobilenet = NeuralNetClassifier(
    # pretrained ResNet50 + custom classifier 
    module=MobileNet,          
    # fine tuning model's inner parameters
    module__output_features=13,
    module__num_units=512,
    module__drop=0.5,
    module__num_units1=512,
    module__drop1=0.5,
    # criterion
    criterion=nn.CrossEntropyLoss,
    # batch_size = 128
    batch_size=20,
    # number of epochs to train
    max_epochs=100,
    # optimizer Adam used
    optimizer=torch.optim.Adam,
    optimizer__lr = 0.0025,
    optimizer__weight_decay=1e-6,
    # shuffle dataset while loading
    iterator_train__shuffle=True,
    # load in parallel
    iterator_train__num_workers=4,
    # stratified kfold split of loaded dataset
    train_split=CVSplit(cv=5, stratified=True, random_state=42),
    # callbacks declared earlier
    callbacks=[lr_scheduler_mobilenet, checkpoint_mobilenet, 
                early_stopping_mobilenet],
    # use GPU or CPU
    device="cuda:0" if torch.cuda.is_available() else "cpu"
)

mobilenet.initialize()

mobilenet.load_params(f_params='/content/drive/MyDrive/chess_weights/best_model4_mobilenet.pkl')

/usr/local/lib/python3.7/dist-packages/skorch/dataset.py:271: FutureWarning: Setting a random_state has no effect since cv is not a float. This will raise an error in a future. You should leave random_state to its default (None), or set cv to a float value.
  FutureWarning


In [13]:
def test_transform():
    return A.Compose([A.Resize(96, 96), A.Normalize(),
    AT.ToTensor()])


class TestDataset(Dataset):
    def __init__(self, image_list, transforms=None):
        super().__init__()
        self.image_list =image_list
        self.transforms = transforms

    def __len__(self) -> int:
        return len(self.image_list)

    def __getitem__(self, index: int):
        image = self.image_list[index]
        if self.transforms:
            sample = {
                'image': image,
            }
            sample = self.transforms(**sample)
            image = sample['image']

        return image

def createDataset(path,batch_size):
    test_dataset = TestDataset(path, test_transform())
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4
    )
    return test_loader

In [37]:
class Detector():
    def __init__(self, device, model, testloader,classes):
        self.model = model
        self.loader = testloader
        self.device = device
        self.classes=classes
        self.figures={0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0}

    def predict(self):
        desc=[]
        images = next(iter(self.loader))
        preds = np.array([])
        predmobile = np.append(preds, self.model.predict(images).tolist())
        for el in predmobile.flatten():
          self.figures[el]+=1
        print('DenseNet169 prediction done!')
        return self.figures,np.array(predmobile).reshape(8,8)

from PIL import Image
from chessboard_detection import inference, loadImage
import matplotlib.pyplot as plt
import numpy as np
import os 

def cropChessboard(im, tiles, folder='resized', padding = 12, n = 50):
    pictures = []
    for tile in tiles:
        xyxy = [min(tile[0:2]), min(tile[2:4]), max(tile[0:2]), max(tile[2:4])]
        xyxy[0] -= padding
        xyxy[1] -= padding
        xyxy[2] += padding
        xyxy[3] += padding
        xyxy = [max(0, int(c)) for c in xyxy]    # preventing negative and float coordinates 
        cropped = im[xyxy[1]:xyxy[3], xyxy[0]:xyxy[2]]
        cropped = Image.fromarray(cropped)

        cropped = cropped.resize((n, n), resample=Image.BILINEAR)
        
        #cropped.save(f'{folder}/{k}.jpg')
        pictures.append(np.array(cropped))
    return pictures # np.array
img_path   = '/content/5.jpg'
imgg = loadImage(img_path)
img= cv2.imread(img_path)
tiles = inference(imgg)
res = cropChessboard(img, tiles)
import torch
import torchvision
import os
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
classes={0:'Empty',1:'whitePawn',2:'whiteBishop',3:'whiteKnight',4:'whiteRook',5:'whiteQueen',6:'whiteKing',7:'blackPawn',8:'blackBishop',9:'blackKnight',10:'blackRook',11:'blackQueen',12:'blackKing'}
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
            


test_loader = createDataset(res, 70)
detector = Detector(device,mobilenet,test_loader,classes)
figures,desc = detector.predict()

/content/chessboard_detection.py:218: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(contours), hierarchy[0]
/content/chessboard_detection.py:200: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  new_contours = new_contours[mask]
/content/chessboard_detection.py:201: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

DenseNet169 prediction done!


In [38]:
figures

{0: 0,
 1: 0,
 2: 57,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 5,
 8: 0,
 9: 1,
 10: 0,
 11: 0,
 12: 1}

In [39]:
fl=desc.flatten()

value= np.bincount(fl.astype('int64')).argmax()
figures.pop(value, None)

57

In [29]:
#desc[desc == value] = 

array([ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  9.,  7., 12.,  2.,  2.,  2.,  7.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  7.,  2.,  2.,  7.,  2.,  2.,  7.,  2.,  2.])

In [33]:
value

2